In [338]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [339]:
train_data_path = "./train.csv"
predict_data_path = "./test.csv"

train_df = pd.read_csv(train_data_path)
predict_df = pd.read_csv(predict_data_path)


In [340]:
#tell apart continuous data and dsicrete data
continuous_columns = ["Id", "LotFrontage", "LotArea", "MasVnrArea", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF" , "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", "LowQualFinSF", "GrLivArea", "BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", "BedroomAbvGr", "KitchenAbvGr", "TotRmsAbvGrd", "Fireplaces", "GarageCars", "GarageArea", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", "ScreenPorch", "PoolArea", "MiscVal", "YearBuilt", "YearRemodAdd", "GarageYrBlt", "SalePrice"]
discrete_columns = []
Id_for_ans = predict_df["Id"].values.tolist()
for i in train_df.columns.values.tolist():
    if (i in continuous_columns)==False:
        discrete_columns.append(i)

In [341]:
#find out the columns that consist of more than half NANs
number_of_data = train_df.shape[0]
number_of_nan = train_df.isna().sum().to_dict()
drop_columns = []
for i in number_of_nan:
    if number_of_nan[i]>=number_of_data/2:
        drop_columns.append(i)

for i in drop_columns:
    if i in continuous_columns:
        continuous_columns.remove(i)
    else:
        discrete_columns.remove(i)

In [342]:
#the final columns that I want to use
select_columns = []

In [343]:
#計算連續型資料的相關係數
continuous_df = train_df[continuous_columns]
corr = continuous_df.corr().values.tolist()
for i, j in zip(continuous_df.columns.values.tolist(), corr[-1]):
    if j>=0.6 and i!="SalePrice":
        select_columns.append(i)

In [345]:
'''
print(select_columns)
for i in select_columns:
    x = [train_df[i]]
    y = train_df["SalePrice"]
    plt.title(i+" vs SalePrice")
    plt.xlabel = i
    plt.ylabel = "SalePrice"
    plt.scatter(x, y)
    plt.show()
'''

'\nprint(select_columns)\nfor i in select_columns:\n    x = [train_df[i]]\n    y = train_df["SalePrice"]\n    plt.title(i+" vs SalePrice")\n    plt.xlabel = i\n    plt.ylabel = "SalePrice"\n    plt.scatter(x, y)\n    plt.show()\n'

In [346]:
train_df = train_df[train_df["TotalBsmtSF"]<=2900]
train_df = train_df[train_df["1stFlrSF"]<=2500]
train_df = train_df[train_df["GrLivArea"]<=3600]
train_df = train_df[train_df["GarageArea"]<=1200]
select_columns.remove("GarageCars")

In [347]:
'''
for i in discrete_columns:
    #x = train_df[i]
    #y = train_df["SalePrice"]
    x = train_df[i].unique().tolist()
    #print(x)
    y = [train_df[train_df[i]==j]["SalePrice"].mean() for j in x]
    #print(y)
    plt.title(i+" vs SalePrice")
    plt.xlabel = i
    plt.ylabel = "SalePrice"
    plt.bar([str(k) for k in x], y)
    plt.show()
'''

'\nfor i in discrete_columns:\n    #x = train_df[i]\n    #y = train_df["SalePrice"]\n    x = train_df[i].unique().tolist()\n    #print(x)\n    y = [train_df[train_df[i]==j]["SalePrice"].mean() for j in x]\n    #print(y)\n    plt.title(i+" vs SalePrice")\n    plt.xlabel = i\n    plt.ylabel = "SalePrice"\n    plt.bar([str(k) for k in x], y)\n    plt.show()\n'

In [348]:

all_std = []
discrete_select_columns = []
for i in discrete_columns:
    x = train_df[i].unique().tolist()
    y = [train_df[train_df[i]==j]["SalePrice"].mean() for j in x]
    s = np.std(y)
    all_std.append(s)

arg = np.argsort(all_std)
for i in range(10):
    discrete_select_columns.append(discrete_columns[arg[len(arg)-i-1]])

discrete_df = train_df[discrete_select_columns]
for i in discrete_select_columns:
    one_hot = pd.get_dummies(discrete_df[i])
    rename_dic = {}
    for j in one_hot.columns.values.tolist():
        rename_dic[j] = str(i)+"_"+str(j)
    one_hot = one_hot.rename(columns=rename_dic)
    discrete_df = discrete_df.drop(i, axis=1)
    discrete_df = discrete_df.join(one_hot)

select_columns = select_columns + discrete_df.columns.values.tolist()
print(select_columns)


['TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'GarageArea', 'BsmtFinType2_ALQ', 'BsmtFinType2_BLQ', 'BsmtFinType2_GLQ', 'BsmtFinType2_LwQ', 'BsmtFinType2_Rec', 'BsmtFinType2_Unf', 'BsmtQual_Ex', 'BsmtQual_Fa', 'BsmtQual_Gd', 'BsmtQual_TA', 'GarageCond_Ex', 'GarageCond_Fa', 'GarageCond_Gd', 'GarageCond_Po', 'GarageCond_TA', 'GarageQual_Ex', 'GarageQual_Fa', 'GarageQual_Gd', 'GarageQual_Po', 'GarageQual_TA', 'GarageFinish_Fin', 'GarageFinish_RFn', 'GarageFinish_Unf', 'GarageType_2Types', 'GarageType_Attchd', 'GarageType_Basment', 'GarageType_BuiltIn', 'GarageType_CarPort', 'GarageType_Detchd', 'FireplaceQu_Ex', 'FireplaceQu_Fa', 'FireplaceQu_Gd', 'FireplaceQu_Po', 'FireplaceQu_TA', 'BsmtCond_Fa', 'BsmtCond_Gd', 'BsmtCond_Po', 'BsmtCond_TA', 'Electrical_FuseA', 'Electrical_FuseF', 'Electrical_FuseP', 'Electrical_Mix', 'Electrical_SBrkr', 'BsmtExposure_Av', 'BsmtExposure_Gd', 'BsmtExposure_Mn', 'BsmtExposure_No']


In [349]:
#計算離散型資料 
'''
discrete_df = train_df[discrete_columns]
for i in discrete_columns:
    one_hot = pd.get_dummies(discrete_df[i])
    rename_dic = {}
    for j in one_hot.columns.values.tolist():
        rename_dic[j] = str(i)+"_"+str(j)
    one_hot = one_hot.rename(columns=rename_dic)
    discrete_df = discrete_df.drop(i, axis=1)
    discrete_df = discrete_df.join(one_hot)

discrete_df.replace(True, 1, inplace=True)
discrete_df.replace(False, 0, inplace=True)

discrete_df = discrete_df.join(train_df["SalePrice"])

corr = discrete_df.corr().values.tolist()
for i, j in zip(discrete_df.columns.values.tolist(), corr[-1]):
    if np.abs(j)>=0.2 and i!="SalePrice":
        select_columns.append(i)
        print(i+" : "+str(j))

'''

'\ndiscrete_df = train_df[discrete_columns]\nfor i in discrete_columns:\n    one_hot = pd.get_dummies(discrete_df[i])\n    rename_dic = {}\n    for j in one_hot.columns.values.tolist():\n        rename_dic[j] = str(i)+"_"+str(j)\n    one_hot = one_hot.rename(columns=rename_dic)\n    discrete_df = discrete_df.drop(i, axis=1)\n    discrete_df = discrete_df.join(one_hot)\n\ndiscrete_df.replace(True, 1, inplace=True)\ndiscrete_df.replace(False, 0, inplace=True)\n\ndiscrete_df = discrete_df.join(train_df["SalePrice"])\n\ncorr = discrete_df.corr().values.tolist()\nfor i, j in zip(discrete_df.columns.values.tolist(), corr[-1]):\n    if np.abs(j)>=0.2 and i!="SalePrice":\n        select_columns.append(i)\n        print(i+" : "+str(j))\n\n'

In [350]:
for i in discrete_select_columns:
    one_hot = pd.get_dummies(train_df[i])
    rename_dic = {}
    for j in one_hot.columns.values.tolist():
        rename_dic[j] = str(i)+"_"+str(j)
    one_hot = one_hot.rename(columns=rename_dic)
    train_df = train_df.drop(i, axis=1)
    train_df = train_df.join(one_hot)

train_df = train_df[select_columns+["SalePrice"]]

In [351]:
def write_ans(predict_target):
    #write the result into ans.csv
    ansDict = {
        "Id" : Id_for_ans,
        "SalePrice" : predict_target
    }
    ansDf = pd.DataFrame(ansDict)
    ansDf.to_csv("ans.csv", index=False)

In [352]:
def data_split(train_df, select_columns):
    #split 20% of the train data to test data
    train_label = train_df[select_columns].values
    train_target = train_df["SalePrice"].values
    x_train, x_test, y_train, y_test = train_test_split(train_label, train_target, test_size=0.2)
    return x_train, x_test, y_train, y_test

In [353]:
def build_test_module(train_df, select_columns):
    x_train, x_test, y_train, y_test = data_split(train_df, select_columns)

    #build a gradient descent module and test whether the columns we choose is good enough
    std = StandardScaler()

    x_train = std.fit_transform(x_train)
    x_test = std.fit_transform(x_test)

    sgd = SGDRegressor()

    sgd.fit(x_train, y_train)
    sgd_pre = sgd.predict(x_test)

    sgd_mean_square = mean_squared_error(y_test, sgd_pre)

    return sgd_mean_square, std, sgd

In [354]:
def build_module(train_df, select_columns):

    #x_train, x_test, y_train, y_test = data_split(train_df, select_columns)

    x_train = train_df[select_columns].values
    y_train = train_df["SalePrice"].values

    #build a gradient descent module and test whether the columns we choose is good enough
    std = StandardScaler()

    x_train = std.fit_transform(x_train)
    #x_test = std.fit_transform(x_test)

    sgd = SGDRegressor()

    sgd.fit(x_train, y_train)
    #sgd_pre = sgd.predict(x_test)

    #sgd_mean_square = mean_squared_error(y_test, sgd_pre)


    return std, sgd

    #0.4 : 612480768.8626099
    #0.2 : 530552347.7190159
    #0.1 : 454339807.24334556
    #0.05 : 433638264.14439946

In [355]:
def predict(std, sgd, predict_df):
    predict_df_columns = predict_df.columns.values.tolist()
    #assume that the module is good enough, predict the data in test.csv with the module
    for i in discrete_select_columns:
        one_hot = pd.get_dummies(predict_df[i])
        rename_dic = {}
        for j in one_hot.columns.values.tolist():
            rename_dic[j] = str(i)+"_"+str(j)
        one_hot = one_hot.rename(columns=rename_dic)
        predict_df = predict_df.drop(i, axis=1)
        predict_df = predict_df.join(one_hot)

    for i in select_columns:
        if (i in predict_df_columns)==False:
            predict_df[i] = [0 for j in range(predict_df.shape[0])]
    
    predict_df = predict_df[select_columns]
    predict_df.replace(True, 1, inplace=True)
    predict_df.replace(False, 0, inplace=True)

    for i in predict_df.columns.values.tolist():
        predict_df[i] = predict_df[i].fillna(predict_df[i].mean())


    predict_label = predict_df.values
    predict_label = std.fit_transform(predict_label)

    predict_target = sgd.predict(predict_label)
    return predict_target

In [356]:
all_min_sqd_mean_square = []
for t in range(1):
    min_sgd_mean_square, min_std, min_sgd = build_test_module(train_df, select_columns)
    for i in tqdm(range(100), desc="predicting "+str(t+1), ncols=100):
        sgd_mean_square, std, sgd = build_test_module(train_df, select_columns)
        if sgd_mean_square<min_sgd_mean_square:
            min_sgd_mean_square = sgd_mean_square
            min_std = std
            min_sgd = sgd
    all_min_sqd_mean_square.append(min_sgd_mean_square)
    #predict_target = predict(min_std, min_sgd, predict_df)
    #write_ans(predict_target)
    print("梯度下降得到的均方误差值: ", min_sgd_mean_square)

print("max : {0}".format(max(all_min_sqd_mean_square))) 
print("min : {0}".format(min(all_min_sqd_mean_square)))
print("avg : {0}".format(np.mean(all_min_sqd_mean_square)))

predicting 1: 100%|██████████████████████████████████████████████| 100/100 [00:00<00:00, 100.53it/s]

梯度下降得到的均方误差值:  799875961.7820473
max : 799875961.7820473
min : 799875961.7820473
avg : 799875961.7820473


In [357]:
std, sgd = build_module(train_df, select_columns)
predict_target = predict(std, sgd, predict_df)
write_ans(predict_target)